In [ ]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [ ]:
# メッセージを格納するリスト
messages=[]

# キャラクター設定
SYSTEM_PROMPT = "あなたは猫のキャラクターとして振る舞うチャットボットです。語尾に「にゃ」を付け、明るく親しみやすく返答してください。"

# システムプロンプトを先頭に追加（キャラクターが消えないように固定）
messages.append({"role": "system", "content": SYSTEM_PROMPT})

try:
    while(True):
        # ユーザーからの質問を受付
        message = input("メッセージを入力:")
        # 質問が入力されなければ終了
        if message.strip()=="":
            break
        print(f"質問:{message}")

        # メッセージにユーザーからの質問を追加
        messages.append({"role": "user", "content": message.strip()})
        # systemを除いた直近8件の会話履歴のみを保持する。超過分はsystemの次から削除する
        if len(messages) > 9:
            del_message = messages.pop(1)

        # APIへリクエスト
        stream = client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages,
            stream=True,
        )

        # 言語モデルからの回答を表示
        response_message = ""
        for chunk in stream:
            if chunk.choices:
                next = chunk.choices[0].delta.content
                if next is not None:
                    response_message += next
                    print(next, end='', flush=True)

        # ストリーム表示の最後に改行を入れる。次の質問が同じ行にくっつかないようにする
        print()

        # メッセージに言語モデルからの回答を追加
        messages.append({"role": "assistant", "content": response_message})

except KeyboardInterrupt:
    print("処理を中断しました。")

print("---ご利用ありがとうございました！---")
